Retrieve Opportunities_examples.csv dataset from DHL.

The columns in this dataset are:
Opp No	Opp Type	Status	Region	Country	BU	Sector	Customer Group	Customer	Product	Created	Cust Dec Date	Go Live Date	Date Exp Gain	Payment Terms	Contract Term	Milestone	 Prob to win % 	BCA_Status	Ann Rev (Ûm)	Ann GP (Ûm)

The model was been generated using Random Forest approach (http://scikit-learn.org/stable/), Pandas (http://pandas.pydata.org/) and Numpy (http://www.numpy.org/).

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import random
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import (accuracy_score,
                             average_precision_score,
                             auc,
                             #auc_score,
                             classification_report,
                             confusion_matrix,
                             explained_variance_score,
                             f1_score,
                             fbeta_score,
                             hamming_loss,
                             hinge_loss,
                             jaccard_similarity_score,
                             log_loss,
                             matthews_corrcoef,
                             mean_squared_error,
                             mean_absolute_error,
                             precision_recall_curve,
                             precision_recall_fscore_support,
                             precision_score,
                             recall_score,
                             r2_score,
                             roc_auc_score,
                             roc_curve,
                             zero_one_loss)
from IPython.display import Image

Sample Opportunities_examples data

In [2]:
train=pd.read_csv('data/Opportunities_examples.csv')
test=pd.read_csv('data/Opportunities_examples.csv')
train['type']='Train' #Create a flag for Train and Test Data set
test['type']='Test'
fullData = pd.concat([train,test],axis=0) #Combined both Train and Test Data set

fullData.columns # This will show all the column names
fullData.head(10) # Show first 10 records of dataframe

,Opp No,Opp Type,Status,Region,Country,BU,Sector,Customer Group,Customer,Product,...,Go Live Date,Date Exp Gain,Payment Terms,Contract Term,Milestone,Prob to win %,BCA_Status,Ann Rev (�m),Ann GP (�m),type
0,9WG2RW,2. Renewal,5. Lost,UK & IRELAND,UK,ELLP,Automotive,FORD MOTOR,Ford Motor,Inbound-to-Manufacturing (I2M),...,01/12/2016,31/12/2016,60,< 1 Year,Closed - Lost,0,Approved,0.33,0.13,Train
1,9YG5FC,1. New,5. Lost,Greater China,China,China,Automotive,FORD MOTOR,Ford Motor,Aftermarket Services,...,29/12/2017,24/11/2017,40,2-3 Years,Closed - Lost,0,Not started,73.59,57.31,Train
2,A4THP4,1. New,2. Opportunity,UK & IRELAND,UK,ELLP,Automotive,FORD MOTOR,Ford Motor,DHL Lead Logistics Partner,...,31/07/2017,31/03/2017,60,2-3 Years,Verbal Customer Commitment Received,90,Awaiting Approval,17.74,35.41,Train
3,A54GJ8,1. New,2. Opportunity,UK & IRELAND,UK,ELLP,Automotive,FORD MOTOR,Ford Motor,DHL Lead Logistics Partner,...,08/01/2018,02/10/2017,60,3-5 Years,Qualified and signed off by Sponsor,20,Not started,176.25,12.14,Train
4,9XY2SC,2. Renewal,2. Opportunity,Greater China,China,China,Automotive,FORD MOTOR,Ford Motor,Warehousing,...,01/01/2017,16/06/2017,60,3-5 Years,Qualified and signed off by Sponsor,85,Awaiting Approval,23.69,18.87,Train
5,244590,1. New,3. Potential Opps,North America,USA,CLA,Automotive,FORD MOTOR,Ford Motor,NaN,...,26/02/2018,02/10/2017,0,1-2 Years,Potential Opportunity,10,NaN,98.12,76.41,Train
6,252136,2. Renewal,3. Potential Opps,UK & IRELAND,UK,ELLP,Automotive,FORD MOTOR,Ford Motor,DHL Lead Logistics Partner,...,01/01/2018,05/05/2017,60,< 1 Year,Potential Opportunity,85,NaN,24.60,24.27,Train
7,256410,2. Renewal,3. Potential Opps,UK & IRELAND,UK,ELLP,Automotive,FORD MOTOR,Ford Otosan,NaN,...,24/07/2017,17/07/2017,0,1-2 Years,Potential Opportunity,85,NaN,0.01,0.00,Train
8,256989,2. Renewal,1. Gain,Greater China,China,China,Automotive,FORD MOTOR,Ford Motor,Warehousing,...,01/12/2015,11/01/2017,45,1-2 Years,Contract Signed,100,Approved,26.32,24.28,Train
9,262296,1. New,7. Cancelled,Greater China,China,China,Automotive,FORD MOTOR,Ford Motor,NaN,...,01/07/2017,30/06/2017,0,2-3 Years,Closed - Canceled,0,NaN,55.19,4.30,Train


We want to do is see what data types we have

In [3]:
fullData.dtypes

Opp No              object
Opp Type            object
Status              object
Region              object
Country             object
BU                  object
Sector              object
Customer Group      object
Customer            object
Product             object
Created             object
Cust Dec Date       object
Go Live Date        object
Date Exp Gain       object
Payment Terms        int64
Contract Term       object
Milestone           object
 Prob to win %       int64
BCA_Status          object
Ann Rev (�m)       float64
Ann GP (�m)        float64
type                object
dtype: object

Summary of numerical fields

In [4]:
fullData.describe()

,Payment Terms,Prob to win %,Ann Rev (�m),Ann GP (�m)
count,130.000000,130.000000,130.000000,130.000000
mean,51.815385,61.123077,67.399231,26.104000
std,27.486370,41.296248,68.009189,25.636583
min,0.000000,0.000000,0.000000,0.000000
25%,35.000000,10.000000,21.480000,9.550000
50%,60.000000,85.000000,42.390000,17.830000
75%,60.000000,100.000000,98.120000,31.930000
max,120.000000,100.000000,306.640000,103.050000


Examples number of each opportunity types

In [5]:
ID_col = ['Opp No']
target_col = ["Status"]
cat_cols = ['Opp Type','Region','Country','BU','Sector', 'Customer Group','Customer','Product','Payment Terms','Contract Term','Milestone','BCA_Status']
num_cols= list(set(list(fullData.columns))-set(cat_cols))
other_col=['type'] #Test and Train Data set identifier

fullData.isnull().any()#Will return the feature with True or False,True means have missing value else False

Opp No             False
Opp Type           False
Status             False
Region             False
Country            False
BU                 False
Sector             False
Customer Group     False
Customer           False
Product             True
Created            False
Cust Dec Date       True
Go Live Date        True
Date Exp Gain      False
Payment Terms      False
Contract Term      False
Milestone          False
 Prob to win %     False
BCA_Status          True
Ann Rev (�m)       False
Ann GP (�m)        False
type               False
dtype: bool

For the sake of simplicity, just fill in the value with the most common value

In [6]:
fullData = fullData.fillna({"Product": "Warehousing"})
fullData = fullData.fillna({"Cust Dec Date": "30/03/2017"})
fullData = fullData.fillna({"Go Live Date": "03/07/2017"})
fullData = fullData.fillna({"BCA_Status": "Approved"})
fullData = fullData.fillna({"Payment Terms": "2-3 Years"})

Combined numerical and Categorical variables.

In [7]:
num_cat_cols = num_cols+cat_cols

Create a new variable for each variable having missing value with VariableName_NA and flag missing value with 1 and other with 0

In [8]:
for var in num_cat_cols:
    if fullData[var].isnull().any()==True:
        fullData[var+'_NA']=fullData[var].isnull()*1

Create label encoders for categorical features

In [11]:
for var in cat_cols:
 number = LabelEncoder()
 fullData[var] = number.fit_transform(fullData[var].astype('str'))

In [14]:
fullData.isnull().any()

Opp No             False
Opp Type           False
Status             False
Region             False
Country            False
BU                 False
Sector             False
Customer Group     False
Customer           False
Product            False
Payment Terms      False
Contract Term      False
Milestone          False
 Prob to win %     False
BCA_Status         False
Ann Rev (�m)       False
Ann GP (�m)        False
type               False
dtype: bool

We using Random Forest Classifier to classify opportunity type. 

In [12]:
fullData = fullData.drop(['Created', 'Cust Dec Date','Go Live Date','Date Exp Gain'], axis=1)

train=fullData[fullData['type']=='Train']
test=fullData[fullData['type']=='Test']

train['is_train'] = np.random.uniform(0, 1, len(train)) <= .75
Train, Validate = train[train['is_train']==True], train[train['is_train']==False]

features=list(set(list(fullData.columns))-set(ID_col)-set(target_col)-set(other_col))

x_train = Train[list(features)].values
y_train = Train["Status"].values
x_validate = Validate[list(features)].values
y_validate = Validate["Status"].values
x_test=test[list(features)].values

random.seed(100)
rf = RandomForestClassifier(n_estimators=1000)
rf.fit(x_train, y_train)

status = rf.predict_proba(x_validate)
fpr, tpr, _ = roc_curve(y_validate, status[:,1], pos_label='T')
roc_auc = auc(fpr, tpr)
print roc_auc

final_status = rf.predict_proba(x_test)
test["Status"]=final_status[:,1]
test.to_csv('data/model_output.csv',columns=['Status'])

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/site-packages/sklearn/metrics/metrics.py:909: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  UndefinedMetricWarning)


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').